# Access to S³I Repository
This is a KWH HMI which allows to access on the cloud date stored in the S³I Repository. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to call up the harvester's data. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests
import collections
from IPython.display import Image
from tools import print_with_timestamp, check_message_encryption, yes, no

## Configure the notebook
In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
hmiId = input('[S3I]: Please enter your HMI id:').strip('," ')
hmiSecret = getpass.getpass('[S3I]: Please enter your HMI secret:').strip('," ')
print_with_timestamp("HMI id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the harvester's data on your behalf.

In [ ]:
print_with_timestamp("DEMO harvester, please log in!")
username = input('[S3I]: Please enter your username:').strip('," ')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmiId,
                                             client_secret=hmiSecret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

After the token has been received, it can be used to query the endpoint of the harvester. To query the harvester in the S³I Directory, you must have the rights to view the harvester. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the harvester, the endpoint of this HMI is also needed to add as "ReplyToEndpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may have changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

## Access to the S³I Repository
In order to get the cloud copy of forest, forest worker, mini tractor operator and mini tractor from the S³I Repository, we need the resptive thing ID. 

In [ ]:
forest = "s3i:c9c55d97-be07-4877-b94e-e68086d861ab"
forest_worker = "s3i:cb8c3169-abf2-4c4c-ab58-6a571f04f984"
mini_tractor = "s3i:e0cec723-1dd9-43be-b9af-c6acea7798f7"
mini_tractor_operator = "s3i:223383b3-52c3-4f92-a5e1-274387501b15"
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
s3i_repo = s3i.Repository("https://ditto.s3i.vswf.dev/api/2/", token=access_token)

### Get the cloud copy of the forest  

In [ ]:
%%html
<img src="images/forest.png", width=70, height=70>

In [ ]:
forest_data = s3i_repo.queryThingIDBased(forest)
print_with_timestamp("You have received the cloud copy of dt forest" + json.dumps(forest_data, indent=2))

### Get the cloud copy of the forest worker

In [ ]:
%%html
<img src="images/forest_worker.png", width=70, height=70>


In [ ]:
forest_worker_data = s3i_repo.queryThingIDBased(forest_worker)
print_with_timestamp("You have received the cloud copy of dt forest worker" + json.dumps(forest_worker_data, indent=2))

### Get the cloud copy of the mini tractor operator

In [ ]:
%%html
<img src="images/mini_tractor_operator.png", width=70, height=70>


In [ ]:
mt_operator_data = s3i_repo.queryThingIDBased(mini_tractor_operator)
print_with_timestamp("You have received the cloud copy of dt mini tractor operator" + json.dumps(mt_operator_data, indent=2))

### Get the cloud copy of the mini tractor 

In [ ]:
%%html
<img src="images/mini_tractor.png", width=70, height=70>


In [ ]:
mt_data = s3i_repo.queryThingIDBased(mini_tractor)
print_with_timestamp("You have received the cloud copy of dt mini_tractor" + json.dumps(mt_data, indent=2))